In [29]:
from pathlib import Path
import xarray as xr
import shlex
import argparse
import hyp3_sdk
from datetime import datetime, timedelta

# Get Inputs from Existing Products

In [2]:
product_paths = sorted(list(Path('existing_products/').glob('*.nc')))
product_paths

[PosixPath('existing_products/S1-GUNW-A-R-064-tops-20240115_20240103-015044-00119W_00033N-PP-9902-v3_0_1.nc'),
 PosixPath('existing_products/S1-GUNW-A-R-064-tops-20240208_20240103-015044-00119W_00033N-PP-629e-v3_0_1.nc'),
 PosixPath('existing_products/S1-GUNW-A-R-064-tops-20240208_20240115-015044-00119W_00033N-PP-8b54-v3_0_1.nc'),
 PosixPath('existing_products/S1-GUNW-A-R-064-tops-20240220_20240115-015044-00119W_00033N-PP-60cc-v3_0_1.nc'),
 PosixPath('existing_products/S1-GUNW-A-R-064-tops-20240220_20240127-015044-00119W_00033N-PP-4f28-v3_0_1.nc'),
 PosixPath('existing_products/S1-GUNW-A-R-064-tops-20240220_20240208-015044-00119W_00033N-PP-e9c9-v3_0_1.nc')]

In [18]:
ids = [p.stem for p in product_paths]
for id_ in ids:
    print(f'{id_},')

S1-GUNW-A-R-064-tops-20240115_20240103-015044-00119W_00033N-PP-9902-v3_0_1,
S1-GUNW-A-R-064-tops-20240208_20240103-015044-00119W_00033N-PP-629e-v3_0_1,
S1-GUNW-A-R-064-tops-20240208_20240115-015044-00119W_00033N-PP-8b54-v3_0_1,
S1-GUNW-A-R-064-tops-20240220_20240115-015044-00119W_00033N-PP-60cc-v3_0_1,
S1-GUNW-A-R-064-tops-20240220_20240127-015044-00119W_00033N-PP-4f28-v3_0_1,
S1-GUNW-A-R-064-tops-20240220_20240208-015044-00119W_00033N-PP-e9c9-v3_0_1,


https://search.asf.alaska.edu/#/?searchType=List%20Search&searchList=S1-GUNW-A-R-064-tops-20240115_20240103-015044-00119W_00033N-PP-9902-v3_0_1,S1-GUNW-A-R-064-tops-20240208_20240103-015044-00119W_00033N-PP-629e-v3_0_1,S1-GUNW-A-R-064-tops-20240208_20240115-015044-00119W_00033N-PP-8b54-v3_0_1,S1-GUNW-A-R-064-tops-20240220_20240115-015044-00119W_00033N-PP-60cc-v3_0_1,S1-GUNW-A-R-064-tops-20240220_20240127-015044-00119W_00033N-PP-4f28-v3_0_1,S1-GUNW-A-R-064-tops-20240220_20240208-015044-00119W_00033N-PP-e9c9-v3_0_1&resultsLoaded=false&granule=S1-GUNW-A-R-064-tops-20240220_20240208-015044-00119W_00033N-PP-e9c9-v3_0_1&zoom=7.062&center=-117.113,33.623

In [3]:
def get_input_scenes_from_command(command: str) -> dict:
    tokens = shlex.split(command)
    
    parser = argparse.ArgumentParser()
    parser.add_argument("--reference-scenes", nargs="+")
    parser.add_argument("--secondary-scenes", nargs="+")
    args, _ = parser.parse_known_args(tokens)
    
    reference_scenes = args.reference_scenes
    secondary_scenes = args.secondary_scenes

    return {'reference': reference_scenes,
            'secondary': secondary_scenes}

def get_input_scenes_from_dataset(path: Path) -> dict:
    with xr.open_dataset(path) as ds:
        scenes = get_input_scenes_from_command(ds.attrs['topsapp_command_line_string'])
    return scenes

In [4]:
scenes_l = [get_input_scenes_from_dataset(p) for p in product_paths]
scenes_l

[{'reference': ['S1A_IW_SLC__1SDV_20240115T015017_20240115T015047_052111_064C5F_5E09',
   'S1A_IW_SLC__1SDV_20240115T015045_20240115T015112_052111_064C5F_91B9'],
  'secondary': ['S1A_IW_SLC__1SDV_20240103T015018_20240103T015047_051936_064669_FC26',
   'S1A_IW_SLC__1SDV_20240103T015045_20240103T015113_051936_064669_08E3']},
 {'reference': ['S1A_IW_SLC__1SDV_20240208T015016_20240208T015046_052461_06582C_8AE1',
   'S1A_IW_SLC__1SDV_20240208T015044_20240208T015112_052461_06582C_5618'],
  'secondary': ['S1A_IW_SLC__1SDV_20240103T015018_20240103T015047_051936_064669_FC26',
   'S1A_IW_SLC__1SDV_20240103T015045_20240103T015113_051936_064669_08E3']},
 {'reference': ['S1A_IW_SLC__1SDV_20240208T015016_20240208T015046_052461_06582C_8AE1',
   'S1A_IW_SLC__1SDV_20240208T015044_20240208T015112_052461_06582C_5618'],
  'secondary': ['S1A_IW_SLC__1SDV_20240115T015017_20240115T015047_052111_064C5F_5E09',
   'S1A_IW_SLC__1SDV_20240115T015045_20240115T015112_052111_064C5F_91B9']},
 {'reference': ['S1A_IW_S

In [8]:
with xr.open_dataset(product_paths[0]) as ds:
    aria_frame_id = ds.attrs['aria_frame_id']
aria_frame_id = int(aria_frame_id)
aria_frame_id

9848

# Submit 

In [21]:
job_dicts = [{'name': 'test-latest-isce',
              # NOTE: we are still using the `dev` branch. Change this to "INSAR_ISCE" to use the `main` branch.
              'job_type': 'INSAR_ISCE',
              'job_parameters': {'granules': scenes['reference'],
                                 'secondary_granules': scenes['secondary'],
                                 'frame_id': aria_frame_id,
                                 'weather_model': 'HRRR',
                                }} 
             for scenes in scenes_l]
job_dicts

[{'name': 'test-latest-isce',
  'job_type': 'INSAR_ISCE',
  'job_parameters': {'granules': ['S1A_IW_SLC__1SDV_20240115T015017_20240115T015047_052111_064C5F_5E09',
    'S1A_IW_SLC__1SDV_20240115T015045_20240115T015112_052111_064C5F_91B9'],
   'secondary_granules': ['S1A_IW_SLC__1SDV_20240103T015018_20240103T015047_051936_064669_FC26',
    'S1A_IW_SLC__1SDV_20240103T015045_20240103T015113_051936_064669_08E3'],
   'frame_id': 9848,
   'weather_model': 'HRRR'}},
 {'name': 'test-latest-isce',
  'job_type': 'INSAR_ISCE',
  'job_parameters': {'granules': ['S1A_IW_SLC__1SDV_20240208T015016_20240208T015046_052461_06582C_8AE1',
    'S1A_IW_SLC__1SDV_20240208T015044_20240208T015112_052461_06582C_5618'],
   'secondary_granules': ['S1A_IW_SLC__1SDV_20240103T015018_20240103T015047_051936_064669_FC26',
    'S1A_IW_SLC__1SDV_20240103T015045_20240103T015113_051936_064669_08E3'],
   'frame_id': 9848,
   'weather_model': 'HRRR'}},
 {'name': 'test-latest-isce',
  'job_type': 'INSAR_ISCE',
  'job_parameter

In [19]:
hyp3_isce = hyp3_sdk.HyP3('https://hyp3-a19-jpl-test.asf.alaska.edu/')

In [22]:
## submitted_jobs = hyp3_isce.submit_prepared_jobs(job_dicts)

In [26]:
print(submitted_jobs)

6 HyP3 Jobs: 0 succeeded, 0 failed, 0 running, 6 pending.


# Track Jobs

In [39]:
import hyp3_sdk

hyp3_isce = hyp3_sdk.HyP3('https://hyp3-a19-jpl-test.asf.alaska.edu/')

now = datetime(2025,9,22,14,0,0)
submit_start = now - timedelta(hours=2)

jobs = hyp3_isce.find_jobs(
                           start=submit_start,
                           user_id='cmarshak',
                           name='test-latest-isce')
print(jobs)

6 HyP3 Jobs: 2 succeeded, 0 failed, 4 running, 0 pending.
